#CSP

## Cryptairthmetic

In [ ]:
import itertools
import re


#Faster Implementation
def compile_formula(formula, verbose=False):
    formula      = formula.replace(' = ', ' == ')
    letters      = cat(sorted(set(re.findall('[A-Z]', formula))))
    firstletters = sorted(set(re.findall(r'\b([A-Z])[A-Z]', formula)))
    body         = re.sub('[A-Z]+', compile_word, formula)
    body         = ' and '.join(firstletters + [body])
    fn = 'lambda {}: {}'.format(','.join(letters), body)
    if verbose: print(fn)
    assert len(letters) <= 10
    return eval(fn), letters

def compile_word(matchobj):
    word = matchobj.group()
    terms = reversed([mult(10**i, L) for (i, L) in enumerate(reversed(word))])
    return '(' + '+'.join(terms) + ')'

def mult(factor, var): return var if factor == 1 else str(factor) + '*' + var


def faster_solve(formula):
    fn, letters = compile_formula(formula)
    for digits in itertools.permutations((1,2,3,4,5,6,7,8,9,0), len(letters)):
        try:
            if fn(*digits):
                yield formula.translate(str.maketrans(letters, cat(map(str, digits))))
        except ArithmeticError: 
            pass

#Standard Solve Implementations
def solve(formula):
    return filter(valid, letter_replacements(formula))

#
def letter_replacements(formula):
    formula = formula.replace(' = ', ' == ') # Allow = or ==
    letters = cat(set(re.findall('[A-Z]', formula)))
    for digits in itertools.permutations('1234567890', len(letters)):
        yield formula.translate(str.maketrans(letters, cat(digits)))

def valid(exp):
    try:
        return not leading_zero(exp) and eval(exp) is True
    except ArithmeticError:
        return False
    
cat = ''.join # Function to concatenate strings
    
leading_zero = re.compile(r'\b0[0-9]').search # Function to check for illegal number
print(next(faster_solve('SEND + MORE = MONEY')))

## Graph Coloring Constraint Library

In [ ]:
from constraint import *

def graph_coloring(graph, num_colors):
    problem = Problem()

    # Create a variable for each vertex and add its domain
    for vertex in graph.keys():
        problem.addVariable(vertex, range(num_colors))

    # Add a constraint for each pair of adjacent vertices
    for vertex, neighbors in graph.items():
        for neighbor in neighbors:
            problem.addConstraint(lambda x, y: x != y, (vertex, neighbor))

    # Find a solution that satisfies all constraints
    solution = problem.getSolutions()

    return solution

# Example usage:
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'C', 'D'],
    'C': ['A', 'B', 'D'],
    'D': ['B', 'C']
}
num_colors = 3

solution = graph_coloring(graph, num_colors)

print(solution) # {'A': 0, 'B': 1, 'C': 2, 'D': 0}


## Graph Coloring

In [ ]:
class CSPSolver:
    def __init__(self, variables, domains, constraints):
        self.variables = variables
        self.domains = domains
        self.constraints = constraints

    def solve(self):
        assignment = {}
        return self.backtrack_search(assignment)

    def backtrack_search(self, assignment):
        if len(assignment) == len(self.variables):
            return assignment

        var = self.select_unassigned_variable(assignment)
        for value in self.order_domain_values(var, assignment):
            if self.is_consistent(var, value, assignment):
                assignment[var] = value
                result = self.backtrack_search(assignment)
                if result is not None:
                    return result
                del assignment[var]
        return None

    def select_unassigned_variable(self, assignment):
        unassigned = set(self.variables) - set(assignment.keys())
        return min(unassigned, key=lambda var: len(self.domains[var]))

    def order_domain_values(self, var, assignment):
        return self.domains[var]

    def is_consistent(self, var, value, assignment):
        assignment[var] = value
        for constraint in self.constraints:
            if not constraint(assignment):
                return False
        return True


# Define variables and domains
variables = ['WA', 'NT', 'Q', 'NSW', 'V', 'SA', 'T']
domains = {
    'WA': ['red', 'green', 'blue'],
    'NT': ['red', 'green', 'blue'],
    'Q': ['red', 'green', 'blue'],
    'NSW': ['red', 'green', 'blue'],
    'V': ['red', 'green', 'blue'],
    'SA': ['red', 'green', 'blue'],
    'T': ['red', 'green', 'blue'],
}

# Define constraints
def adjacent_regions_must_have_different_colors(assignment):
    if 'WA' in assignment and 'NT' in assignment:
        if assignment['WA'] == assignment['NT']:
            return False
    # Define similar constraints for other adjacent regions
    return True

def no_constraint_violations(assignment):
    return True

constraints = [adjacent_regions_must_have_different_colors, no_constraint_violations]

# Create the CSPSolver instance and solve the problem
solver = CSPSolver(variables, domains, constraints)
solution = solver.solve()
print(solution)


## N Queens

In [ ]:
class CSPSolver:
    def __init__(self, variables, domains, constraints):
        self.variables = variables
        self.domains = domains
        self.constraints = constraints

    def solve(self):
        assignment = {}
        return self.backtrack_search(assignment)

    def backtrack_search(self, assignment):
        if len(assignment) == len(self.variables):
            return assignment

        var = self.select_unassigned_variable(assignment)
        for value in self.order_domain_values(var, assignment):
            if self.is_consistent(var, value, assignment):
                assignment[var] = value
                result = self.backtrack_search(assignment)
                if result is not None:
                    return result
                del assignment[var]
        return None

    def select_unassigned_variable(self, assignment):
        unassigned = set(self.variables) - set(assignment.keys())
        return min(unassigned, key=lambda var: len(self.domains[var]))

    def order_domain_values(self, var, assignment):
        return self.domains[var]

    def is_consistent(self, var, value, assignment):
        # Check row and column constraints
        if value in assignment.values():
            return False
        for k, v in assignment.items():
            if abs(k - var) == abs(v - value):
                return False
        return True

def print_solution(solution):
    if solution is None:
        print("No solution found")
    else:
        for i in range(len(solution)):
            row = ""
            for j in range(len(solution)):
                if solution[i] == j:
                    row += "Q "
                else:
                    row += ". "
            print(row)

# Define variables and domains
n = 8  # Change n to the desired value of N
variables = list(range(n))
domains = {i: list(range(n)) for i in range(n)}

# Define constraints
constraints = []

# Create the CSPSolver instance and solve the problem
solver = CSPSolver(variables, domains, constraints)
solution = solver.solve()
print(solution)
print_solution(solution)


## Job Scheduling Constraint Libary

In [ ]:
from constraint import *

# Define the job scheduling problem
problem = Problem()

# Define the variables and domains
# Each variable represents a job, and the domain of each variable is the time required to complete the job
job1 = [3, 5, 1]
job2 = [2, 4, 6]
job3 = [4, 3, 2]
problem.addVariables(["job1", "job2", "job3"], [job1, job2, job3])

# Define the constraints
# Each constraint enforces that no two jobs overlap in time
for i in range(3):
    for j in range(i+1, 3):
        problem.addConstraint(lambda j1, j2: j1[i]+j1[j] <= j2[i] or j2[i]+j2[j] <= j1[i], ("job"+str(i+1), "job"+str(j+1)))

# Solve the problem
solutions = problem.getSolutions()

# Print the solutions
for solution in solutions:
    print(solution)

## TSP Constraint Libary

In [ ]:
from constraint import Problem, AllDifferentConstraint

# Define the distance matrix between cities
distances = [
    [0, 10, 15, 20],
    [10, 0, 35, 25],
    [15, 35, 0, 30],
    [20, 25, 30, 0]
]

# Define the number of cities
n = len(distances)

# Create a problem instance
problem = Problem()

# Create variables for the order in which cities are visited
order = list(range(n))
problem.addVariables(order, range(n))

# Add constraint to ensure each city is visited exactly once
problem.addConstraint(AllDifferentConstraint(), order)

# Define a function to calculate the total distance of a given route
def calculate_distance(route):
    distance = 0
    for i in range(n-1):
        distance += distances[route[i]][route[i+1]]
    distance += distances[route[n-1]][route[0]]
    return distance

# Add constraint to minimize the total distance of the route
problem.addConstraint(lambda *route: calculate_distance(route), order)

# Solve the problem
solution = problem.getSolution()

# Print the solution
print("Shortest route:", [solution[i] for i in range(n)])
print("Total distance:", calculate_distance([solution[i] for i in range(n)]))


## Latin Probelm Constraint Library

In [ ]:
from constraint import Problem, AllDifferentConstraint

# Define the size of the Latin square
n = 5

# Create a problem instance
problem = Problem()

# Create variables for each cell in the Latin square
variables = []
for i in range(n):
    for j in range(n):
        variables.append((i, j))

# Add domain constraints for each variable
for variable in variables:
    problem.addVariable(variable, range(1, n+1))

# Add row constraints
for i in range(n):
    row_variables = [(i, j) for j in range(n)]
    problem.addConstraint(AllDifferentConstraint(), row_variables)

# Add column constraints
for j in range(n):
    column_variables = [(i, j) for i in range(n)]
    problem.addConstraint(AllDifferentConstraint(), column_variables)

# Add diagonal constraints
diagonal1_variables = [(i, i) for i in range(n)]
problem.addConstraint(AllDifferentConstraint(), diagonal1_variables)

diagonal2_variables = [(i, n-i-1) for i in range(n)]
problem.addConstraint(AllDifferentConstraint(), diagonal2_variables)

# Solve the problem
solutions = problem.getSolutions()

# Print the solutions
for solution in solutions:
    for i in range(n):
        for j in range(n):
            print(solution[(i, j)], end=' ')
        print()
    print()

#Adversial

##Tic Tac Toe Min Max

In [ ]:
class TicTacToe:
    def __init__(self):
        self.board = [" " for x in range(9)]
        self.player = "X"

    def display_board(self):
        row1 = "| {} | {} | {} |".format(self.board[0], self.board[1], self.board[2])
        row2 = "| {} | {} | {} |".format(self.board[3], self.board[4], self.board[5])
        row3 = "| {} | {} | {} |".format(self.board[6], self.board[7], self.board[8])

        print()
        print(row1)
        print(row2)
        print(row3)

    def player_move(self):
        print("Player {} turn.".format(self.player))
        choice = int(input("Enter your move (1-9): ").strip())
        if self.board[choice - 1] == " ":
            self.board[choice - 1] = self.player
        else:
            print("That space is already taken.")
            self.player_move()

    def check_for_winner(self):
        winning_combinations = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # rows
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # columns
            [0, 4, 8], [2, 4, 6]  # diagonals
        ]

        for combination in winning_combinations:
            if self.board[combination[0]] == self.board[combination[1]] == self.board[combination[2]] != " ":
                return self.board[combination[0]]

        if " " not in self.board:
            return "Tie"

        return None

    def minimax(self, player):
        result = self.check_for_winner()

        if result is not None:
            if result == "X":
                return -1
            elif result == "O":
                return 1
            else:
                return 0

        if player == "X":
            best = [None, -2]
            for i in range(9):
                if self.board[i] == " ":
                    self.board[i] = "X"
                    score = self.minimax("O")
                    self.board[i] = " "
                    if score > best[1]:
                        best = [i, score]
        else:
            best = [None, 2]
            for i in range(9):
                if self.board[i] == " ":
                    self.board[i] = "O"
                    score = self.minimax("X")
                    self.board[i] = " "
                    if score < best[1]:
                        best = [i, score]

        return best[1]

    def computer_move(self):
        print("Computer turn.")
        move = self.minimax("O")
        print(move)
        if move == 1:
            for i in range(9):
                if self.board[i] == " ":
                    self.board[i] = "O"
                    if self.check_for_winner() == "O":
                        print("Computer chooses {}".format(i + 1))
                        return
                    self.board[i] = " "

        elif move == -1:
            for i in range(9):
                if self.board[i] == " ":
                    self.board[i] = "X"
                    if self.check_for_winner() == "X":
                        self.board[i] = "O"
                        print("Computer chooses {}".format(i + 1))
                        return
                    self.board[i] = " "

        for i in range(9):
            if self.board[i] == " ":
                self.board[i] = "O"
                print("Computer chooses {}".format(i + 1))
                break

game = TicTacToe()
while True:
    game.display_board()
    game.player_move()
    result = game.check_for_winner()
    if result is not None:
        game.display_board()
        if result == "Tie":
            print("Tie game.")
        else:
            print("{} wins!".format(result))
        break
    game.computer_move()
    result = game.check_for_winner()
    if result is not None:
        game.display_board()
        if result == "Tie":
            print("Tie game.")
        else:
            print("{} wins!".format(result))
        break


##MinMax

In [ ]:
# Returns optimal value for current player
# (Initially called for root and maximizer)
def minimax(depth, nodeIndex, maximizingPlayer, values):
    # Terminating condition. i.e
    # leaf node is reached
    if depth == 3:
        return values[nodeIndex]

    if maximizingPlayer:
        best = -float('inf')

        # Recur for left and right children
        for i in range(0, 2):
            val = minimax(depth + 1, nodeIndex * 2 + i,
                          False, values)
            best = max(best, val)

        return best
    else:
        best = float('inf')

        # Recur for left and
        # right children
        for i in range(0, 2):
            val = minimax(depth + 1, nodeIndex * 2 + i,
                          True, values)
            best = min(best, val)

        return best

# Driver Code
if __name__ == "__main__":
    values = [2, 4, 6, 8, 1, 2, 10, 12]
    print("The optimal value is:", minimax(0, 0, True, values))


##Alpha Beta Pruning

In [ ]:
# Initial values of Alpha and Beta
MAX, MIN = 1000, -1000

# Returns optimal value for current player
# (Initially called for root and maximizer)
def minimax(depth, nodeIndex, maximizingPlayer,
            values, alpha, beta):
    # Terminating condition. i.e
    # leaf node is reached
    if depth == 3:
        return values[nodeIndex]

    if maximizingPlayer:
        best = MIN

        # Recur for left and right children
        for i in range(0, 2):
            val = minimax(depth + 1, nodeIndex * 2 + i,
                          False, values, alpha, beta)
            best = max(best, val)
            alpha = max(alpha, best)

            # Alpha Beta Pruning
            if beta <= alpha:
                break

        return best
    else:
        best = MAX

        # Recur for left and
        # right children
        for i in range(0, 2):
            val = minimax(depth + 1, nodeIndex * 2 + i,
                          True, values, alpha, beta)
            best = min(best, val)
            beta = min(beta, best)

            # Alpha Beta Pruning
            if beta <= alpha:
                break

        return best

# Driver Code
if __name__ == "__main__":
    values = [2, 4, 6, 8, 1, 2, 10, 12]
    print("The optimal value is:", minimax(0, 0, True, values, MIN, MAX))

#Linear Regression

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

data = pd.read_csv('data.csv')
X = data[['x1', 'x2', ... , 'xn']] # independent variables
y = data['y'] # dependent variable

model = LinearRegression()
model.fit(X, y)

model = LinearRegression()
model.fit(X, y)

predictions = model.predict(new_data)

#Least Squares
slope, intercept = np.polyfit(x, y, 1)

# Evaluation Metrics

In [ ]:
#R2 score
r2 = r2_score(y_actual, y_pred)
print('R-squared score:', r2)

#MSE & RSE  
rmse = np.sqrt(mean_squared_error(y_actual, y_pred))
rse = np.sqrt(sum((y_pred - y_actual)**2) / (len(y_actual) - 2))
print('RMSE:', rmse)
print('RSE:', rse)

#accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score
acc = accuracy_score(y_test, preds)
print('Accuracy:', acc)

#ROC
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score
probs = clf.predict_proba(X_test)[:, 1]

# Calculate false positive rate, true positive rate, and thresholds for ROC curve
fpr, tpr, thresholds = roc_curve(y_test, probs)

# Plot ROC curve
plt.plot(fpr, tpr, linestyle='--', label='Logistic Regression')

# Add labels and legend to plot
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()


#confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true, y_pred)


# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)


# K means Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Create the classifier object
clf = KNeighborsClassifier(n_neighbors=5)

# Train the model on the data
clf.fit(X_train, y_train)

# Predict the output for the test data
y_pred = clf.predict(X_test)


# SVM

In [ ]:
from sklearn.svm import SVC

# Create the classifier object
clf = SVC(kernel='linear', C=1.0)


# Train the model on the data
clf.fit(X_train, y_train)

# Predict the output for the test data
y_pred = clf.predict(X_test)


svm = SVC(kernel='rbf')

# train the SVM classifier
svm.fit(X_train, y_train)

# make predictions on test set
y_pred = svm.predict(X_test)


# KMeans


In [ ]:
kmeans = KMeans(n_clusters=2)

# fit the K-Means model on the data
kmeans.fit(X)

# get the coordinates of the centroids
centroids = kmeans.cluster_centers_

# get the labels of each point
labels = kmeans.labels_


#Reinforcement Learning

##Frozen Lake

In [ ]:
import numpy as np
import gym

#setting environment
env = gym.make('FrozenLake-v1')


#set hyperparams
alpha = 0.1  
gamma = 0.99
num_episodes = 10000 #doubled the training 

#initialize Q table
num_states = env.observation_space.n
num_actions = env.action_space.n


Q = np.zeros((env.observation_space.n,env.action_space.n))

#run Q learning algorithm now
for i in range(num_episodes):
    #reset env
    state = env.reset()

    done = False

    total_reward = 0


    while not done:
        if np.random.uniform(0,1) < 0.5:
            action = env.action_space.sample()

        else:
            action = np.argmax(Q[state,:])

        #take action and observe next state and reward

        next_state,reward,done,info = env.step(action)

        # Update the Q-value of the (state, action) pair

        Q[state,action] = Q[state,action] + alpha *(reward + gamma *np.max(Q[next_state,:])-Q[state,action])

        state = next_state
        total_reward += reward

     
    print(f"Episode {i}: Total reward = {total_reward}") 


#tesing
num_test_episodes = 500 #increased from 100 to 500
num_test_steps = 500 #increased from 100 to 500
num_successes = 0


for i in range(num_test_episodes):
    state=env.reset()
    done = False
    steps = 0


    while not done and steps < num_test_steps:
        action = np.argmax(Q[state,:])

        next_state,reward,done,info = env.step(action)

        state=next_state
        steps+=1

    if state==15:
        num_successes+=1

print("Success rate:", num_successes/num_test_episodes) 

## Traffic v3

In [ ]:
env = gym.make('Taxi-v3')
Q = np.zeros((env.observation_space.n, env.action_space.n))
learning_rate = 0.8
discount_factor = 0.95
num_episodes = 5000
max_steps = 100

for episode in range(num_episodes):
    state = env.reset()
    total_reward = 0
    done = False
    
    for step in range(max_steps):
        epsilon = 0.1  
        if np.random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() 
        else:
            action = np.argmax(Q[state, :])

        next_state, reward, done, _ = env.step(action)

        Q[state, action] = Q[state, action] + learning_rate * (
                reward + discount_factor * np.max(Q[next_state, :]) - Q[state, action])

        total_reward += reward
        state = next_state

        if done:
            break

    print("Episode:", episode, "Total Reward:", total_reward)

num_eval_episodes = 100
eval_rewards = []

for episode in range(num_eval_episodes):
    state = env.reset()
    total_reward = 0
    done = False

    while not done:
        action = np.argmax(Q[state, :])
        next_state, reward, done, _ = env.step(action)
        total_reward += reward
        state = next_state

    eval_rewards.append(total_reward)

print("Average Evaluation Reward:", np.mean(eval_rewards))


##Task 3 RL

In [ ]:
num_nodes = 5
sink_node = 0
cost_matrix = np.array([[0, 1, 2, 3, 4],
                        [1, 0, 1, 2, 3],
                        [2, 1, 0, 1, 2],
                        [3, 2, 1, 0, 1],
                        [4, 3, 2, 1, 0]])

Q = np.zeros((num_nodes, num_nodes))

learning_rate = 0.8
discount_factor = 0.95
num_episodes = 2000
max_steps = 100

for episode in range(num_episodes):
    state = np.random.randint(0, num_nodes)  
    total_cost = 0
    done = False
    
    for step in range(max_steps):
        epsilon = 0.1 
        if np.random.uniform(0, 1) < epsilon:
            action = np.random.randint(0, num_nodes)  
        else:
            action = np.argmax(Q[state, :])  

        cost = cost_matrix[state, action]

        Q[state, action] = Q[state, action] + learning_rate * (
                cost + discount_factor * np.min(Q[action, :]) - Q[state, action])

        total_cost += cost
        state = action

        if state == sink_node:
            break

    print("Episode:", episode, "Total Cost:", total_cost)

num_eval_episodes = 10
eval_costs = []

for episode in range(num_eval_episodes):
    state = np.random.randint(0, num_nodes) 
    total_cost = 0
    done = False

    while not done:
        action = np.argmax(Q[state, :])
        cost = cost_matrix[state, action]
        total_cost += cost
        state = action

        if state == sink_node:
            break

    eval_costs.append(total_cost)

print("Average Evaluation Cost:", np.mean(eval_costs))


## Epsilon Greedy

In [ ]:
#another way for Q-learning 

import numpy as np
import gym

# Create the FrozenLake-v0 environment
env = gym.make('FrozenLake-v1')

# Set the hyperparameters
num_episodes = 10000
max_steps = 100
alpha = 0.1
gamma = 0.99
epsilon = 1.0
epsilon_min = 0.01
epsilon_decay = 0.99

# Initialize the Q-table
num_states = env.observation_space.n
num_actions = env.action_space.n
Q = np.zeros((num_states, num_actions))

# Define the epsilon-greedy policy
def epsilon_greedy_policy(state, epsilon):
    if np.random.rand() < epsilon:
        # Take a random action
        action = env.action_space.sample()
    else:
        # Choose the best action from the Q-table
        action = np.argmax(Q[state, :])
    return action

# Loop over episodes
for episode in range(num_episodes):
    # Reset the environment and get the initial state
    state = env.reset()
    
    # Choose the initial action using the epsilon-greedy policy
    action = epsilon_greedy_policy(state, epsilon)
    
    # Initialize the total reward for the episode
    total_reward = 0
    
    # Loop over steps within this episode
    for t in range(max_steps):
        # Take the chosen action and observe the next state and reward
        next_state, reward, done, info = env.step(action)
        
        # Choose the next action using the epsilon-greedy policy
        next_action = epsilon_greedy_policy(next_state, epsilon)
        
        # Update the Q-table
        td_error = reward + gamma * Q[next_state, next_action] - Q[state, action]
        Q[state, action] += alpha * td_error
        
        # Update the state, action, and total reward
        state = next_state
        action = next_action
        total_reward += reward
        
        # If the episode is complete, break out of the loop
        if done:
            break
    
    # Decay the epsilon value for the next episode
    epsilon = max(epsilon_min, epsilon * epsilon_decay)
    
    # Print the total reward for this episode
    print(f"Episode {episode}: Total reward = {total_reward}")
    
# Print the final Q-table
print("Final Q-table:")
print(Q)


## SARSA

In [ ]:
import numpy as np
import gym

# FrozenLake-v0 gym environment
env = gym.make('FrozenLake-v1')

# Parameters
epsilon = 0.9
total_episodes = 10000
max_steps = 100
alpha = 0.05
gamma = 0.95
  
#Initializing the Q-vaue
Q = np.zeros((env.observation_space.n, env.action_space.n))

# Function to choose the next action with episolon greedy
def choose_action(state):
    action=0
    if np.random.uniform(0, 1) < epsilon:
        action = env.action_space.sample()
    else:
        action = np.argmax(Q[state, :])
    return action
    
#Initializing the reward
reward=0
  
# Starting the SARSA learning
for episode in range(total_episodes):
    t = 0
    state1 = env.reset()
    action1 = choose_action(state1)
  
    while t < max_steps:
        # Visualizing the training
#         env.render()
          
        # Getting the next state
        state2, reward, done, info = env.step(action1)
  
        #Choosing the next action
        action2 = choose_action(state2)
          
        #Learning the Q-value
        Q[state1, action1] = Q[state1, action1] + alpha * (reward + gamma * Q[state2, action2] - Q[state1, action1])
  
        state1 = state2
        action1 = action2
          
        #Updating the respective vaLues
        t += 1
        reward += 1
          
        #If at the end of learning process
        if done:
            break
            
#Evaluating the performance
print ("Performace : ", reward/total_episodes)
  
#Visualizing the Q-matrix
print(Q)

## Cart Pole

In [ ]:
!pip install stable-baselines3
!pip install pyglet
############################
import gym 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

###########################

environment_name = "CartPole-v0"
env = gym.make(environment_name)
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

#########################
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1)
model.learn(total_timesteps=20000)


##########################

obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    #env.render()
    if done: 
        print('info', info)
        break
env.close()



# Bayes Net

In [ ]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD

# Define the Bayesian network structure
model = BayesianModel([('guest', 'prize'), ('choice', 'prize'), ('choice', 'guest')])

# Define the conditional probability tables (CPDs) for each node
guest_cpd = TabularCPD(variable='guest', variable_card=3, values=[[1/3, 1/3, 1/3]])
choice_cpd = TabularCPD(variable='choice', variable_card=3, values=[[1/3, 1/3, 1/3]])
prize_cpd = TabularCPD(variable='prize', variable_card=3,
                       evidence=['guest', 'choice'], evidence_card=[3, 3],
                       values=[[0, 0, 0, 0, 0.5, 1, 0, 1, 0.5],
                               [0.5, 0, 0.5, 1, 0, 0, 0.5, 0, 0.5],
                               [0.5, 1, 0.5, 0, 0.5, 0, 0.5, 0, 0]])

# Add the CPDs to the model
model.add_cpds(guest_cpd, choice_cpd, prize_cpd)

# Check if the model is valid
model.check_model()

# Calculate the probabilities for winning the prize
result = model.query(variables=['prize'], evidence={'choice': 1, 'guest': 2})
print(result['prize'])

In [ ]:
import matplotlib.pyplot as plt
import math
from pomegranate import *

study_time = DiscreteDistribution({'1hour': 0.5, '2hour': 0.4, '3hour': 0.1})

iq = DiscreteDistribution({'100 iq': 0.2, '110 iq': 0.6, '120 iq': 0.2, })

# Define the conditional probability table
exam_score = ConditionalProbabilityTable(
    [        # Study Time: 1 hour        # IQ: 100        
        ['1hour', '100 iq', 'Poor', 0.2],
        ['1hour', '100 iq', 'Average', 0.5],
        ['1hour', '100 iq', 'Excellent', 0.3],
        ['1hour', '110 iq', 'Poor', 0.15],
        ['1hour', '110 iq', 'Average', 0.6],
        ['1hour', '110 iq', 'Excellent', 0.25],
        ['1hour', '120 iq', 'Poor', 0.1],
        ['1hour', '120 iq', 'Average', 0.5],
        ['1hour', '120 iq', 'Excellent', 0.4],
        # Study Time: 2 hours
        # IQ: 100
        ['2hour', '100 iq', 'Poor', 0.1],
        ['2hour', '100 iq', 'Average', 0.4],
        ['2hour', '100 iq', 'Excellent', 0.5],
        ['2hour', '110 iq', 'Poor', 0.05],
        ['2hour', '110 iq', 'Average', 0.5],
        ['2hour', '110 iq', 'Excellent', 0.45],
        ['2hour', '120 iq', 'Poor', 0.02],
        ['2hour', '120 iq', 'Average', 0.3],
        ['2hour', '120 iq', 'Excellent', 0.68],
        # Study Time: 3 hours
        # IQ: 100
        ['3hour', '100 iq', 'Poor', 0.05],
        ['3hour', '100 iq', 'Average', 0.3],
        ['3hour', '100 iq', 'Excellent', 0.65],
        ['3hour', '110 iq', 'Poor', 0.02],
        ['3hour', '110 iq', 'Average', 0.4],
        ['3hour', '110 iq', 'Excellent', 0.58],
        ['3hour', '120 iq', 'Poor', 0.01],
        ['3hour', '120 iq', 'Average', 0.25],
        ['3hour', '120 iq', 'Excellent', 0.74]
    ], 
    [study_time, iq]
)


pass_exam = ConditionalProbabilityTable(
    [[ 'Poor', False, 1],
     [ 'Poor', True, 0],
     [ 'Average', False, 0.5],
     [ 'Average', True, 0.5],
     [ 'Excellent', True, 1],
     [ 'Excellent', False, 0],
     ],
    [exam_score])

d1 = State(study_time, name='study_time')
d2 = State(iq, name='iq')
d3 = State(exam_score, name='exam_score')
d4 = State(pass_exam, name='pass_exam')

network = BayesianNetwork()
network.add_states(d1,d2,d3,d4)
network.add_edge(d1,d3)
network.add_edge(d2,d3)
network.add_edge(d3,d4)

network.bake()
beliefs = network.predict_proba({'study_time': '1hour', 'iq':'100 iq'})
print("n".join( "{}t{}".format( state.name, str(belief) ) for state, belief in zip( network.states, beliefs )))

In [ ]:
age = DiscreteDistribution({'<40': 1./3, '40-60': 1./3, '>60': 1./3})
cholestrol = DiscreteDistribution({'100-150': 1./3, '150-250': 1./3, '250-255': 1./3 })

heart_disease = ConditionalProbabilityTable(
    [        ['<40', '100-150', 'No', 0.75],
        ['<40', '100-150', 'Yes', 0.25],
        ['<40', '150-250', 'No', 0.4],
        ['<40', '150-250', 'Yes', 0.6],
        ['<40', '250-255', 'No', 0.2],
        ['<40', '250-255', 'Yes', 0.8],
        ['40-60', '100-150', 'No', 0.37],
        ['40-60', '100-150', 'Yes', 0.63],
        ['40-60', '150-250', 'No', 0.37],
        ['40-60', '150-250', 'Yes', 0.63],
        ['40-60', '250-255', 'No', 0.12],
        ['40-60', '250-255', 'Yes', 0.88],
        ['>60', '100-150', 'No', 0.375],
        ['>60', '100-150', 'Yes', 0.625],
        ['>60', '150-250', 'No', 0.5],
        ['>60', '150-250', 'Yes', 0.5],
        ['>60', '250-255', 'No', 0.375],
        ['>60', '250-255', 'Yes', 0.625],
    ], 
    [age, cholestrol]
)
d1 = State(age, name='age')
d2 = State(cholestrol, name='cholestrol')
d3 = State(heart_disease, name='heart_disease')

network = BayesianNetwork()
network.add_states(d1,d2,d3)
network.add_edge(d1,d3)
network.add_edge(d2,d3)
network.bake()
beliefs = network.predict_proba({'age': '40-60', 'cholestrol':'100-150'})
print("n".join( "{}t{}".format( state.name, str(belief) ) for state, belief in zip( network.states, beliefs )))

In [ ]:
from pomegranate import *

#dist for parent nodes
cpu = DiscreteDistribution({"Low": 0.1, "Medium": 0.2, "High": 0.5})
memory = DiscreteDistribution({"Low": 0.1, "Medium": 0.2, "High": 0.5})
disk = DiscreteDistribution({"Low": 0.1, "Medium": 0.2, "High": 0.5})
network = DiscreteDistribution({"Low": 0.1, "Medium": 0.2, "High": 0.5})
OS = DiscreteDistribution({"Windows": 0.5, "Linux": 0.5})

#CPT for app probabilities

lcpu = ["Low","Medium","High"]
lmemory = ["Low","Medium","High"]
ldisk = ["Low","Medium","High"]
lnetwork = ["Low","Medium","High"]
lOS = ["Windows","Linux"]
lapp = ["Desktop","Mobile"]
count=0
table=list()
for i in lcpu:
    for j in lmemory:
        for k in ldisk:
            for l in lnetwork:
                for m in lOS:
                    for n in lapp:
                        #print(i,j,k,l,m,n,"0.0")
                        rec=[i,j,k,l,m,n,0.5]
                        table.append(rec)
                        count+=1

app = ConditionalProbabilityTable(table,[cpu,memory,disk,network,OS])

failure = ConditionalProbabilityTable(
    [["Desktop","pass",0.6],
     ["Desktop","fail",0.4],
     ["Mobile","pass",0.7],
     ["Mobile","fail",0.3]],
    [app])

# Create the Bayesian Network object and add the nodes to the network
model = BayesianNetwork("Failure Predictorr")

#Creating the nodes for the events
nfailure = Node(failure, name="failure")
napp = Node(app, name="app")
ndisk = Node(disk, name="disk")
nnetwork = Node(network, name="network")
nmemory = Node(memory, name="memory")
nOS = Node(OS, name="OS")
ncpu = Node(cpu, name="cpu")

model.add_states(nfailure,napp,ncpu,nmemory,ndisk,nnetwork,nOS)

# Add edges between the nodes to represent the dependencies between them
model.add_edge(ncpu,napp)
model.add_edge( nmemory,napp)
model.add_edge( ndisk,napp)
model.add_edge( nOS,napp)
model.add_edge( nnetwork,napp)
model.add_edge( napp,nfailure)


# Finalize the network structure and start the model
model.bake()

# Probability of sample events leading to system failure
observations={'cpu': 'High', 'memory': 'Low', 'disk': 'Low', 'network': 'Low', 'OS': 'Windows'}
#print(model.predict_proba({"failure": "pass"}, evidence=observations))
print(model.predict_proba({"failure": "pass", **observations}))

#Task 1 Linear Regression

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

#Read
df1 = pd.read_csv(r'C:\Users\Mohsin\Desktop\Dataset_Q1.csv')
df1.head()
#Dataframe
X = df1[['Size (sq ft)']]
y = df1['Price (USD)']


from sklearn.model_selection import train_test_split

# split the data into training and test sets with a 70/30 ratio
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

from sklearn.linear_model import LinearRegression

#Learning
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)  # make predictions
# evaluate the model
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
mse = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print('Mean Absolute error:', mse)
print('R2 score:', r2)

#Task 2 Decision Tree

In [ ]:
#File Read
df2 = pd.read_csv(r'C:\Users\Mohsin\Desktop\Dataset_Q2.csv')
df2.head()

#Encoding
categorical_cols = df2.select_dtypes(include=['object'])
from sklearn.preprocessing import LabelEncoder

for col in categorical_cols:
    le = LabelEncoder()
    df2[col] = le.fit_transform(df2[col])

df2
#Dataframe
X = df2.drop(columns=['Customer ID','Churn','Phone Service'])
y = df2.iloc[:,-1]

#Spliting
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.3,random_state=42)


#Decision Tree Classifier
dtc = DecisionTreeClassifier(criterion = 'entropy', max_depth=10, random_state=42)
dtc.fit(X_train,y_train)
y_pred = dtc.predict(X_test)
score = accuracy_score(y_pred, y_test)
print(score)

#Task 3 KNNeighbours


In [ ]:
df3 = pd.read_csv(r'C:\Users\Mohsin\Desktop\Dataset_Q3.csv')
df3.head()

#Dataframe
X = df3.iloc[:,1:-1]
y = df3.iloc[:,-1]
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

#Odd Points so take square root of 9 and always take odd points i.e sqrt(144)=12 but take 11
knn = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
knn.fit(x_train,y_train)
y_pred = knn.predict(x_test)


score = accuracy_score(y_test, y_pred)
mat = confusion_matrix(y_pred, y_test)
print('Score:', score)
print(mat)

#plot
sns.scatterplot(data=x_train, x='Sepal Length', y='Petal Width', hue=y_train)
sns.scatterplot(data=x_test, x='Sepal Length', y='Petal Width',  hue=y_pred, palette='Reds')

#Task 4 SVM

In [ ]:
#Read
df4 = pd.read_csv(r'C:\Users\Mohsin\Desktop\Dataset_Q4.csv')
df4.head()

#Encode
df4.drop(columns='ID',inplace=True)
cat_cols = df4.select_dtypes(include=['object'])

for col in cat_cols:
    enc = LabelEncoder()
    df4[col] = enc.fit_transform(df4[col])
    
print(df4)
#plot
sns.pairplot(df4,hue='Fraudulent')

#Dataframe
X = df4.drop(columns=['Fraudulent'])
y = df4['Fraudulent']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)
from sklearn.svm import SVC

svc = SVC(kernel='linear')
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
print(accuracy_score(y_pred, y_test))

## Task 5 Kmeans

In [ ]:
#Read
df5 = pd.read_csv(r'C:\Users\Mohsin\Desktop\Dataset_Q5.csv')
df5.head()

#Encoding
cat_cols = df5.select_dtypes(include=['object'])

for col in cat_cols:
    enc = LabelEncoder()
    df5[col] = enc.fit_transform(df5[col])
    
#Dataframe    
X = df5.drop(columns=[' Items Purchased'])

from sklearn.cluster import KMeans

plt.figure(figsize=(5,5))
wss = []
for i in range(1,6):
    kmeans = KMeans(n_clusters=i, init='k-means++' ,random_state=42)
    kmeans.fit_predict(X)
    wss.append(kmeans.inertia_)

plt.plot(range(1,6),wss)
plt.title("The Elbow Method")
plt.xlabel("Number of clusters")
plt.show()

from sklearn.metrics import silhouette_score

for i in range(2,7):
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(X)
    score = silhouette_score(X,kmeans.labels_)
    print(f"for cluster {i}: The Score is {score}")
    
km = KMeans(n_clusters=6, init='k-means++')
y_means = km.fit_predict(X)


plt.scatter(X.loc[:,' Age'], X.loc[:,' Total Spending'], c=km.labels_)

plt.xlabel('Age')
plt.ylabel('Total Spending')
plt.show()